# Proper Orthogonal Decomposition for Reduced Order Modelling 

<a id="table"></a>

## Table of contents
- [Introduction](#intro)
- [High resolution model](#HR)
- [Proper orthogonal decomposition](#POD)
- [Low resolution model](#LR)
- [Performance improvement](#improvement)
- [Error between high and low resolution models](#error)

In [1]:
from time import process_time
import numpy as np
from scipy import linalg
#from matplotlib import pyplot as plt

# ROM functions
#from modules.rom import rom
# Two phase, two layers, reservoir model
from scripts import water_canals_model as model
from modules.simulator import simplots

# Set image folder path
imgpath = ".\\images"

<a id="intro"></a>

## [Introduction](#table)

 Reduced order modeling (ROM) has the aim of reducing the number of dimensions (equations) that represent a model in order to solve it faster. In this notebook I will show an example of how to do ROM with proper orthogonal decomposition (POD).

<a id="HR"></a>

## [High resolution model (HRM)](#table)

For this example the model is a single layer of rock of low permeability with two canals of high permeability. There is a single phase flow of water caused by a difference of pressure between the left face and right face of the rock. [This image](#perm) shows the permeability distribution.

The model has a grid with 10,000 cells and it is solved with the Lagging Coefficients method. So for each timestep a linear system with dimensions 10,000 x 10,000 is solved.

The simulation is run fur 200 timesteps equivalent to 2 days. Solving the HRM takes approximately **175 seconds** to run.

In [2]:
model.grid.cellnumber

10000

In [3]:
# Plot permeability
log_perm = np.log(model.rock.perm/model.u.darcy / model.u.milli)
simplots.plotCellValues2D(model.grid, log_perm, 'inferno', np.min(log_perm), np.max(log_perm),
                     title='Permeability Ln(mD)', 
                     filename='{}\\pod_perm'.format(imgpath))

<a id="perm">
        <img src=".\images\pod_perm.png" style="width:400px;height:-1px;" >
</a> 

In [17]:
# Load the reservoir model 
#exec(open(".\\scripts\\reservoir_model_01.py").read())
sch  = model.sch
# Run simulation
t = process_time()
# Run simulation
r, well_solution, sch = model.LCM.solve(model.sch, max_inner_iter = 2, tol = 1E-6, ATS = False)
elapsed_time = process_time() - t
model.nice_print('Time running simulation = {:.3f} seconds.'.format(elapsed_time))

 Time running simulation = 181.679 seconds.

In [13]:
# Transform units to psi
p = r/model.u.psi
# We will  save a plot for each timestep and then use 
# ffmpeg to create a gif
acum_sum = np.cumsum(model.sch.timesteps) / model.u.day
# Add "0" for the initial conditions
acum_sum = np.hstack((0, acum_sum))

for k in np.arange(0,acum_sum.size, 1): 
    model.plotCellValues2D(model.grid, p[:, k], 'inferno', np.min(p), np.max(p),
                     title='Pressure (psi). {:.3f} days'.format(acum_sum[k]), 
                     filename='{}\\pod_pw_{}'.format(imgpath, k))

The following animation shows the pressure distribution in the rock. As it is expected, the pressure increases in the high permeability area of the canals.

<img src=".\images\pod_pw.gif" >

<a id = POD></a>

## [Proper orthogonal decomposition](#table)

In [ ]:
# Singular value decomposition
_,S_po,_ = linalg.svd(results['p_oil'][:,:50])
_,S_sw,_ = linalg.svd(results['sw'][:,:50])

In [ ]:
rom.plot_energy(S_po, "{}\\po_energy".format(imgpath))
rom.plot_energy(S_sw, "{}\\sw_energy".format(imgpath))

**Oil pressure basis vectors**
<img src = ".\images\po_energy.png" style="width:400px;height:-1px;" >
**Water saturation basis vectors**
<img src = ".\images\sw_energy.png" style="width:400px;height:-1px;" >

In [ ]:
from modules.rom import rom
x = results['p_oil'][:,:50]
y = results['sw'][:,:50]
Ur_po, Sr_po = rom.pod(x, 99.999, 6)
Ur_sw, Sr_sw = rom.pod(y, 99.99, 6)

Plot the 

In [ ]:
rom.plot_basis(Ur_po[:100,:], 2, 3, model.grid, "{}\\basis_po_1st".format(imgpath))
rom.plot_basis(Ur_sw[:100,:], 2, 3, model.grid, "{}\\basis_sw_1st".format(imgpath))

<img src = ".\images\basis_po_1st.png" height="1" >
<img src = ".\images\basis_sw_1st.png" height="1" >

<a id="LR"></a>

## [Low resoludion model](#table)

Now, we will apply the proper orthogonal decomposition technique to the reservoir simulator to speed up the simulation. We will modify the Fully Implicit Solver. After generating the linear syustem to be solved we project it to create the low order model and then solve it with the neton raphson model.
The dimensions of the basis is  x by y so the dimensions of the new system will be k by w. 

In [ ]:
sch  = rmodel.sch
# Run simulation
t = process_time()
results, well_solution, sch, info = rmodel.FIM.solve(sch, max_iter= 10, tol = 1E-6, ATS = False)
elapsed_time = process_time() - t
print('Time running simulation = {:.3f} seconds.'.format(elapsed_time))#

<a id="improvement"></a>

## [Performance improvement](#table)



### Time improvement

<a id="error"></a>

## [Error between high and low resolution models](#table)

The error between the high resolution model and the low resolution model

<a id="bottom"></a>

## [To the top](#table)

In [15]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# Using the style sheet found here  Lorena Barba /* https://github.com/barbagroup/CFDPython/blob/master/styles/custom.css */